<a href="https://colab.research.google.com/github/RaphaelRAY/airbnb-rating-ml/blob/main/notebooks/01.5_feature_engineering_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 02 - Feature Engineering Geográfica para Dados Airbnb - Rio de Janeiro

Este notebook demonstra o processo de enriquecimento do dataset de listings do Airbnb com features geográficas, como distâncias a pontos de interesse (POIs) e contagem de POIs próximos. Utilizamos o arquivo `listings_processed.csv` (que é o `airbnb_rio_clean.csv` da etapa anterior) como base e o `neighbourhoods.geojson` para contexto geográfico, embora os POIs sejam baixados via OpenStreetMap (OSM) para maior granularidade.

**Nota:** Para evitar timeouts em ambientes com recursos limitados, este notebook executa a feature engineering em uma **amostra** do dataset. Para processar o dataset completo, as linhas de amostragem devem ser comentadas no script `02_feature_engineering.py`.

## 1. Configuração Inicial e Carregamento de Dados

Importação das bibliotecas necessárias e carregamento do dataset `listings_processed.csv`.

In [2]:
import pandas as pd
import os
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon
%pip install osmnx
import osmnx as ox

# Definir o diretório de saída
output_dir = "data/processed"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Diretório \'{output_dir}\' criado.")

print("\n--- Carregando o dataset processado de listings ---")
# Usar o arquivo listings_processed.csv fornecido pelo usuário
df = pd.read_csv("https://github.com/RaphaelRAY/airbnb-rating-ml/raw/refs/heads/main/data/processed/listings_Encode.csv")
print(f"Dataset carregado. Shape inicial: {df.shape}")


--- Carregando o dataset processado de listings ---
Dataset carregado. Shape inicial: (41724, 40)


## 2. Módulo `geo_features.py`

As funções para baixar POIs, calcular distâncias (Haversine) e adicionar features geográficas foram encapsuladas no módulo `src/airbnb_rating/utils/geo_features.py` para melhor organização e reusabilidade. Este módulo inclui otimizações como o uso de índices espaciais para acelerar os cálculos.

In [3]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
import os

EARTH_RADIUS_KM = 6371

def get_pois(city: str, geojson_path: str = None):
    tags = {
        "natural": "beach",
        "aeroway": "aerodrome",
        "tourism": True,
        "amenity": ["restaurant", "bar", "cafe"],
        "leisure": ["park"]
    }
    print(f"Baixando POIs para {city} do OpenStreetMap...")
    pois = ox.features_from_place(city, tags)
    return pois.to_crs("EPSG:4326")

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return EARTH_RADIUS_KM * c

def add_geo_features(df, city="Rio de Janeiro, Brazil", geojson_path: str = None):
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df["longitude"], df["latitude"]), crs="EPSG:4326"
    )
    pois = get_pois(city, geojson_path)
    beaches = pois[pois["natural"] == "beach"]
    airports = pois[pois["aeroway"] == "aerodrome"]
    tourism = pois[pois["tourism"].notna()]
    restaurants = pois[pois["amenity"].isin(["restaurant", "bar", "cafe"]) if "amenity" in pois.columns else False]
    parks = pois[pois["leisure"] == "park"]

    beaches_sindex = beaches.sindex
    airports_sindex = airports.sindex
    tourism_sindex = tourism.sindex
    restaurants_sindex = restaurants.sindex
    parks_sindex = parks.sindex

    def get_poi_coords(geom):
        if geom is None or not geom.is_valid:
            return np.nan, np.nan
        if geom.geom_type == 'Point':
            return geom.y, geom.x
        elif geom.geom_type == 'Polygon':
            return geom.centroid.y, geom.centroid.x
        return np.nan, np.nan

    def min_dist_optimized(point_geom, targets_gdf, targets_sindex):
        if targets_gdf.empty:
            return np.nan
        possible_matches_indices = list(targets_sindex.intersection(point_geom.bounds))
        if not possible_matches_indices:
            return np.nan
        min_d = np.inf
        for idx in possible_matches_indices:
            target_geom = targets_gdf.iloc[idx].geometry
            target_lat, target_lon = get_poi_coords(target_geom)
            if not np.isnan(target_lat):
                dist = haversine(point_geom.y, point_geom.x, target_lat, target_lon)
                if dist < min_d:
                    min_d = dist
        return min_d if min_d != np.inf else np.nan

    print("Calculando distância para praias...")
    gdf["dist_beach_km"] = gdf.geometry.apply(lambda p: min_dist_optimized(p, beaches, beaches_sindex))
    print("Calculando distância para aeroportos...")
    gdf["dist_airport_km"] = gdf.geometry.apply(lambda p: min_dist_optimized(p, airports, airports_sindex))
    print("Calculando distância para pontos turísticos...")
    gdf["dist_touristic_km"] = gdf.geometry.apply(lambda p: min_dist_optimized(p, tourism, tourism_sindex))

    def count_nearby_optimized(point_geom, targets_gdf, targets_sindex, radius_km=1):
        if targets_gdf.empty:
            return 0
        count = 0
        radius_deg = radius_km / 111.139
        buffered_point = point_geom.buffer(radius_deg)
        possible_matches_indices = list(targets_sindex.intersection(buffered_point.bounds))

        for idx in possible_matches_indices:
            target_geom = targets_gdf.iloc[idx].geometry
            target_lat, target_lon = get_poi_coords(target_geom)
            if not np.isnan(target_lat):
                if haversine(point_geom.y, point_geom.x, target_lat, target_lon) <= radius_km:
                    count += 1
        return count

    print("Contando restaurantes próximos...")
    gdf["n_restaurants_1km"] = gdf.geometry.apply(lambda p: count_nearby_optimized(p, restaurants, restaurants_sindex, radius_km=1))
    print("Contando parques próximos...")
    gdf["n_parks_2km"] = gdf.geometry.apply(lambda p: count_nearby_optimized(p, parks, parks_sindex, radius_km=2))

    gdf = pd.DataFrame(gdf.drop(columns="geometry"))
    return gdf

In [ ]:
print("\n--- Adicionando features geográficas automáticas ---")
df_enriched = add_geo_features(df, city="Rio de Janeiro, Brazil") # Usar
print(f"Dataset enriquecido. Novo shape: {df_enriched.shape}")


--- Adicionando features geográficas automáticas ---
Baixando POIs para Rio de Janeiro, Brazil do OpenStreetMap...
Calculando distância para praias...
Calculando distância para aeroportos...


## 4. Salvando o Dataset Enriquecido

O dataset resultante, contendo as novas features geográficas, é salvo em um novo arquivo CSV.

In [ ]:
print("\n--- Salvando o dataset enriquecido ---")
df_enriched.to_csv(os.path.join(output_dir, "airbnb_rio_geo_sample.csv"), index=False) # Salvar amostra
print("✅ Dataset enriquecido (amostra) salvo em data/processed/airbnb_rio_geo_sample.csv")